In [ ]:
!pip install bcolz==1.2.1

     |████████████████████████████████| 1.5 MB 7.4 MB/s 
  Created wheel for bcolz: filename=bcolz-1.2.1-cp37-cp37m-linux_x86_64.whl size=838590 sha256=1a70c46460c52ad20a762f482a0362204734dffebeeb0e76f983617712500e4c
  Stored in directory: /root/.cache/pip/wheels/2c/35/ca/9d914de345914e2446ea285170329f771b8abba2a00f7650bd
Successfully built bcolz
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import bcolz
import numpy as np
import pickle
from helper.word2vec import Word2Vec
from helper.text_cleaning import clean, remove_stop
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
w2v = Word2Vec() 

FileNotFoundError: [Errno 2] No such file or directory: 'glove/6B.50.dat'

In [ ]:
# cleans up line, splits it, and removes stop words
def get_valuable_tokens(line):
    line = clean(line)
    line = line.split(' ')
    line = [w for w in line if w]
    line = remove_stop(line)

    return line

In [ ]:
demo_tokens = get_valuable_tokens("Late night when you need my love Call me on my cell phone And I know when that hotline bling\n")
print(demo_tokens)

['late', 'night', 'need', 'love', 'call', 'cell', 'phone', 'know', 'hotline', 'bling']


In [ ]:
def vectorize(tokens):
    w2vecs = [(token, w2v.v(token)) for token in tokens]
    tokens = []
    for token, w2vec in w2vecs:
        # if w2vec is None:
            # fixes = [
            #     (token.endswith("d"), token[:-1] + "ed")
            # ]
            # for (cond, fix) in fixes:
            #     if w2vec is not None:
            #         break
            #     if cond:
            #         w2vec = w2v.v(fix)
        
        if w2vec is not None:
            tokens.append(w2vec)
        else:
            print(f"warning: {token} has no w2vec repr")

    assert not any([x is None for x in tokens]), tokens
    return tokens

In [ ]:
demo_vectors = vectorize(demo_tokens)
print(len(demo_vectors), demo_vectors[0].shape)
print(demo_vectors)

10 (50,)
[array([ 0.21994 , -0.014414, -0.24502 , -0.31895 , -0.32942 , -0.48162 ,
       -1.5818  ,  0.27079 , -1.3613  ,  0.05855 , -0.10478 , -0.44319 ,
       -0.40213 ,  0.05586 ,  0.81842 ,  0.30685 , -1.0009  , -0.8302  ,
       -1.2259  ,  0.54736 ,  0.60423 ,  0.33345 ,  0.02349 , -0.58298 ,
        0.068311, -1.4961  ,  0.12819 , -0.18686 , -0.17151 ,  0.90003 ,
        3.1905  , -0.59917 , -0.097557, -0.021908,  0.54973 , -0.14088 ,
        0.18978 ,  0.34743 , -0.061865, -0.067381, -0.5258  ,  0.55705 ,
       -0.5649  , -0.42131 ,  0.24299 ,  0.1408  , -0.074287, -0.41503 ,
       -0.17553 , -0.62826 ]), array([ 3.0814e-01,  4.7129e-01, -2.7929e-01,  3.7760e-01,  2.5881e-01,
       -6.4276e-01, -1.0796e+00,  9.0681e-01, -5.7363e-01, -2.3595e-01,
       -9.0416e-01, -8.6146e-01, -4.9546e-01,  6.7405e-01,  9.5942e-01,
       -1.1408e-01, -5.3341e-01, -3.1823e-01, -1.6635e+00, -3.7480e-01,
        2.2628e-01,  1.0046e+00,  3.7941e-01,  5.6968e-02,  6.0677e-02,
       -1.2871e

def group_vectors(vectors, group_size=5):
    output = []
    for i in range(len(vectors) // 5):
        output.append(np.array([v for v in vectors[i*5:(i+1)*5]]))
    return output

In [ ]:
demo_groups = group_vectors(demo_vectors)
print(len(demo_groups), demo_groups[0].shape)
print(demo_groups)

2 (5, 50)
[array([[ 2.1994e-01, -1.4414e-02, -2.4502e-01, -3.1895e-01, -3.2942e-01,
        -4.8162e-01, -1.5818e+00,  2.7079e-01, -1.3613e+00,  5.8550e-02,
        -1.0478e-01, -4.4319e-01, -4.0213e-01,  5.5860e-02,  8.1842e-01,
         3.0685e-01, -1.0009e+00, -8.3020e-01, -1.2259e+00,  5.4736e-01,
         6.0423e-01,  3.3345e-01,  2.3490e-02, -5.8298e-01,  6.8311e-02,
        -1.4961e+00,  1.2819e-01, -1.8686e-01, -1.7151e-01,  9.0003e-01,
         3.1905e+00, -5.9917e-01, -9.7557e-02, -2.1908e-02,  5.4973e-01,
        -1.4088e-01,  1.8978e-01,  3.4743e-01, -6.1865e-02, -6.7381e-02,
        -5.2580e-01,  5.5705e-01, -5.6490e-01, -4.2131e-01,  2.4299e-01,
         1.4080e-01, -7.4287e-02, -4.1503e-01, -1.7553e-01, -6.2826e-01],
       [ 3.0814e-01,  4.7129e-01, -2.7929e-01,  3.7760e-01,  2.5881e-01,
        -6.4276e-01, -1.0796e+00,  9.0681e-01, -5.7363e-01, -2.3595e-01,
        -9.0416e-01, -8.6146e-01, -4.9546e-01,  6.7405e-01,  9.5942e-01,
        -1.1408e-01, -5.3341e-01, -3.18

In [ ]:
with open('data/shake/raw.txt', 'r') as shake:
    all_lines= []
    for line in shake.readlines():
        all_lines.append(line)

    all_lines = list(set(all_lines))
    
    # GET LIST OF SHAKE SNAPSHOTS

In [ ]:
with open('data/drake/clean.txt', 'r') as drake:
    all_lines= []
    for line in drake.readlines():
        all_lines.append(line)

    all_lines = list(set(all_lines))
    
    # GET LIST OF DRAKE SNAPSHOTS

In [ ]:
# COMBINE INTO TRAINING AND TEST SETS

In [ ]:
# DEFINE MODEL
class ShakeOrDrake(nn.Module):
    def __init__(self):
        pass

    def forward(self, x):
        pass

In [ ]:
# TRAIN MODEL

In [ ]:
# TEST MODEL

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d19f7349-666c-452b-af43-7a0d9fe413f0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>